# # Práctica de Laboratorio de Procesamiento del Lenguaje Natural (NLP)
Tema: Word2Vec


Entrenar un modelo de word2vec es una tarea muy costosa computacionalmente, que además requiere de corpus de texto muy grandes, del orden de miles de millones de palabras. Afortunadamente existen modelos word2vec pre-entrenados que están disponibles de forma pública y con los que podemos trabajar para hacer nuestros análisis semánticos. Uno de ellos es el modelo SBWC, disponible aquí: http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz. Añádelo a la carpeta que contiene este Notebook para poder cargarlo.


## Similitud semántica

Un modelo pre-entrenado word2vec no es más que un diccionario en el que para cada palabra tenemos el vector que la representa. Podemos cargar en memoria este diccionario utilizando el paquete de análisis de texto gensim.

In [1]:
# Importamos las librerías

import gensim
from gensim.models import Word2Vec,KeyedVectors

In [3]:
# Importamos el modelo Word2Vec

model = KeyedVectors.load_word2vec_format('models/SBW-vectors-300-min5.bin.gz',binary=True)

Podemos comprobar ahora la representación vectorial de diferentes palabras, por ejemplo:

In [4]:
# Cómo es un vector

vec = model['hombre']
print(vec)

[ 0.00120981 -0.29563493  0.16428724  0.07638286  0.1308095  -0.18907565
 -0.19585267 -0.18845399  0.18023378 -0.10991588  0.10583372 -0.20523688
 -0.27889782  0.08500005 -0.01454076  0.07790862 -0.19201109 -0.2023368
  0.08294028  0.3402465  -0.08416018 -0.2755257  -0.0714149   0.0808465
 -0.13162656 -0.14391989  0.0958087   0.39754403  0.1138585   0.23643324
  0.12858306 -0.17455381 -0.04746418 -0.01137695  0.14309211  0.08217653
  0.02318554  0.08878922 -0.04167972  0.07108709  0.01710395 -0.02570473
 -0.12259014 -0.02195755 -0.08128569  0.19045328 -0.06413727  0.12798432
 -0.05037531  0.10685983  0.35144055  0.29801247 -0.03209334  0.20625433
  0.07129728  0.31428477 -0.11478377  0.15143426 -0.2149212   0.16646026
 -0.3328038   0.10436062  0.13380383  0.1914494  -0.03352465  0.19137044
 -0.29684854  0.11710161 -0.23980273 -0.24970213 -0.143163   -0.30643377
  0.08513806  0.18788363 -0.28649864  0.37120643  0.00868526  0.10604249
 -0.01570174 -0.10231962 -0.02137911 -0.14277178 -0.0

Entre otras funcionalidades, el modelo permite localizar aquellos términos más similares al especificado:

In [5]:
# Similitudes

model.most_similar('perro',topn=5)

[('perros', 0.7533276677131653),
 ('cachorro', 0.753067135810852),
 ('gato', 0.7445882558822632),
 ('schnauzer', 0.7370451092720032),
 ('mastín', 0.7347643375396729)]

Aunque la representación vectorial de una palabra resulta oscura de interpretar, se ha comprobado que sigue una lógica semántica y sintáctica. Esto permite hacer aritmética con estos vectores y obtener resultados que son coherentes con lo que cabría esperar. Por ejemplo, podemos encontrar casos como: 

`king` - `man` + `woman` = `queen`

Podemos comprobar que eso es cierto utilizando la función most_similar del objeto que contiene los embeddings. Esta función recibe dos listas de palabras, a contribuir de forma positiva o negativa a la operación aritmética, y devuelve las palabras cuya representación vectorial sea más cercana al vector resultado de la operación, ordenadas por similitud. Utiliza el parámetro `positive` para añadir los términos que quieras sumar, y `negative` para restar. 

### 1. ¿Cuánto es rey + mujer - hombre?


In [6]:
# Analogías

model.most_similar(positive=["rey","mujer"],negative=["hombre"],topn=3)

[('reina', 0.7493031620979309),
 ('consorte', 0.7034263014793396),
 ('princesa', 0.6861541867256165)]


### 2. ¿Cuánto es Madrid + Francia - España?

In [8]:
# Madrid - España + Francia

model.most_similar(positive=["Madrid","Francia"],negative=["España"],topn=3)

[('París', 0.7476180791854858),
 ('Lyon', 0.7073330879211426),
 ('Marsella', 0.6997580528259277)]

### 3. ¿Cómo conseguir "hermano" utilizando "hermana"?

In [15]:
model.most_similar(positive=['hermana', 'él'], negative=['ella'])

[('hermano', 0.8807544708251953),
 ('sobrino', 0.7946146726608276),
 ('hijo', 0.7890192270278931),
 ('tío', 0.7820650339126587),
 ('padre', 0.7775164246559143),
 ('cuñado', 0.776477038860321),
 ('sobrina', 0.7542605996131897),
 ('abuelo', 0.7538495063781738),
 ('hija', 0.753645658493042),
 ('nieto', 0.7477919459342957)]

### 4. Busca al menos 2 ejemplos interesantes para compartir con la clase. ¡Los vamos a comentar!

### 4. ¿Cuál es la similitud entre la palabra perro y la palabra gato? ¿Y entre perro y lápiz?

Utiliza la función `similarity` para calcularlo; obtendrás un valor entre 0 y 1, donde 1 indicará la máxima similitud.

In [8]:
# Como de próximas están varias palabras

print("perro - gato: ", model.similarity("perro", "gato"))
print("perro - lápiz: ", model.similarity("perro", "lápiz"))

perro - gato:  0.74458814
perro - lápiz:  0.33357212


Una utilidad interesante de las distancias semánticas entre palabras es de resolver los típicos pasatiempos en los que se debe identificar la palabra que no encaja dentro de un grupo dado. Por ejemplo:

group = ["Biden", "Obama", "Bush", "camión"]

Está claro que camión es la palabra intrusa en esta lista de presidentes de EEUU. Pero esto es algo que sabemos por nuestro amplio conocimiento del mundo y del lenguaje, y para un programa informático no es nada trivial llegar a esta conclusión. Sin embargo gracias a las representaciones semánticas en forma de vector que nos da word2vec podemos hacerlo.

### 5. Utiliza la función `doesnt_match` para comprobar que de la lista: televisión, tostadora, lavavajillas y montaña, montaña es la palabra que no encaja.


In [31]:
model.doesnt_match(["televisión", "tostadora", "lavavajillas", "montaña"])

'montaña'